In [3]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import h5py

train_data_path = 'data/train/'
test_data_path = 'data/test/'

def save_bottleneck_features(MODEL, image_size, module_name, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory(train_data_path, image_size, shuffle=False, batch_size=16)
    test_generator = gen.flow_from_directory(test_data_path, image_size, shuffle=False, batch_size=16, class_mode=None)

    train = model.predict_generator(train_generator)
    test = model.predict_generator(test_generator)
    with h5py.File("bottleneck_features/bottleneck_v3.h5") as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

#save_bottleneck_features(ResNet50, (224, 224), 'ResNet50')
#save_bottleneck_features(InceptionV3, (299, 299), 'InceptionV3', inception_v3.preprocess_input)
#save_bottleneck_features(Xception, (299, 299), 'Xception', xception.preprocess_input)

In [1]:
import h5py
import numpy as np
from sklearn.utils import shuffle
from keras.models import *
from keras.layers import *

np.random.seed(2017)

X_train = []
X_test = []

#for filename in ["bottleneck_features/gap_ResNet50.h5", "bottleneck_features/gap_Xception.h5", "bottleneck_features/gap_InceptionV3.h5"]:
for filename in ["bottleneck_features/bottleneck_v3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)

X_train, y_train = shuffle(X_train, y_train)

np.random.seed(2017)

print(X_train.shape[1:])

input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])


model.fit(X_train, y_train, batch_size=128, epochs=8, validation_split=0.2)

c:\users\tracy\anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


(2048,)
Train on 16000 samples, validate on 4000 samples
Epoch 1/8
16000/16000 [==============================] - 2s 140us/step - loss: 0.1623 - acc: 0.9574 - val_loss: 0.0487 - val_acc: 0.9892
Epoch 2/8
16000/16000 [==============================] - 1s 54us/step - loss: 0.0473 - acc: 0.9893 - val_loss: 0.0265 - val_acc: 0.9942
Epoch 3/8
16000/16000 [==============================] - 1s 58us/step - loss: 0.0367 - acc: 0.9896 - val_loss: 0.0196 - val_acc: 0.9942
Epoch 4/8
16000/16000 [==============================] - 1s 62us/step - loss: 0.0305 - acc: 0.9913 - val_loss: 0.0171 - val_acc: 0.9945
Epoch 5/8
16000/16000 [==============================] - 1s 57us/step - loss: 0.0277 - acc: 0.9921 - val_loss: 0.0154 - val_acc: 0.9942
Epoch 6/8
16000/16000 [==============================] - 1s 49us/step - loss: 0.0276 - acc: 0.9918 - val_loss: 0.0148 - val_acc: 0.9945
Epoch 7/8
16000/16000 [==============================] - 1s 55us/step - loss: 0.0260 - acc: 0.9921 - val_loss: 0.0144 - val_ac

In [7]:
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("data/sample_submission.csv")

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory(test_data_path, (224, 224), shuffle=False, batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    print(fname, fname.rfind('\\'), fname.rfind('.'))
    index = int(fname[fname.rfind('\\')+1:fname.rfind('.')])
    print(index)
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv('data/pred.csv', index=None)
df.head(10)

12500/12500 [==============================] - 0s 23us/step
Found 12500 images belonging to 1 classes.
none\1.jpg 4 6
1
none\10.jpg 4 7
10
none\100.jpg 4 8
100
none\1000.jpg 4 9
1000
none\10000.jpg 4 10
10000
none\10001.jpg 4 10
10001
none\10002.jpg 4 10
10002
none\10003.jpg 4 10
10003
none\10004.jpg 4 10
10004
none\10005.jpg 4 10
10005
none\10006.jpg 4 10
10006
none\10007.jpg 4 10
10007
none\10008.jpg 4 10
10008
none\10009.jpg 4 10
10009
none\1001.jpg 4 9
1001
none\10010.jpg 4 10
10010
none\10011.jpg 4 10
10011
none\10012.jpg 4 10
10012
none\10013.jpg 4 10
10013
none\10014.jpg 4 10
10014
none\10015.jpg 4 10
10015
none\10016.jpg 4 10
10016
none\10017.jpg 4 10
10017
none\10018.jpg 4 10
10018
none\10019.jpg 4 10
10019
none\1002.jpg 4 9
1002
none\10020.jpg 4 10
10020
none\10021.jpg 4 10
10021
none\10022.jpg 4 10
10022
none\10023.jpg 4 10
10023
none\10024.jpg 4 10
10024
none\10025.jpg 4 10
10025
none\10026.jpg 4 10
10026
none\10027.jpg 4 10
10027
none\10028.jpg 4 10
10028
none\10029.jpg 4 

c:\users\tracy\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  app.launch_new_instance()


 10
10544
none\10545.jpg 4 10
10545
none\10546.jpg 4 10
10546
none\10547.jpg 4 10
10547
none\10548.jpg 4 10
10548
none\10549.jpg 4 10
10549
none\1055.jpg 4 9
1055
none\10550.jpg 4 10
10550
none\10551.jpg 4 10
10551
none\10552.jpg 4 10
10552
none\10553.jpg 4 10
10553
none\10554.jpg 4 10
10554
none\10555.jpg 4 10
10555
none\10556.jpg 4 10
10556
none\10557.jpg 4 10
10557
none\10558.jpg 4 10
10558
none\10559.jpg 4 10
10559
none\1056.jpg 4 9
1056
none\10560.jpg 4 10
10560
none\10561.jpg 4 10
10561
none\10562.jpg 4 10
10562
none\10563.jpg 4 10
10563
none\10564.jpg 4 10
10564
none\10565.jpg 4 10
10565
none\10566.jpg 4 10
10566
none\10567.jpg 4 10
10567
none\10568.jpg 4 10
10568
none\10569.jpg 4 10
10569
none\1057.jpg 4 9
1057
none\10570.jpg 4 10
10570
none\10571.jpg 4 10
10571
none\10572.jpg 4 10
10572
none\10573.jpg 4 10
10573
none\10574.jpg 4 10
10574
none\10575.jpg 4 10
10575
none\10576.jpg 4 10
10576
none\10577.jpg 4 10
10577
none\10578.jpg 4 10
10578
none\10579.jpg 4 10
10579
none\1058.j

none\10995.jpg 4 10
10995
none\10996.jpg 4 10
10996
none\10997.jpg 4 10
10997
none\10998.jpg 4 10
10998
none\10999.jpg 4 10
10999
none\11.jpg 4 7
11
none\110.jpg 4 8
110
none\1100.jpg 4 9
1100
none\11000.jpg 4 10
11000
none\11001.jpg 4 10
11001
none\11002.jpg 4 10
11002
none\11003.jpg 4 10
11003
none\11004.jpg 4 10
11004
none\11005.jpg 4 10
11005
none\11006.jpg 4 10
11006
none\11007.jpg 4 10
11007
none\11008.jpg 4 10
11008
none\11009.jpg 4 10
11009
none\1101.jpg 4 9
1101
none\11010.jpg 4 10
11010
none\11011.jpg 4 10
11011
none\11012.jpg 4 10
11012
none\11013.jpg 4 10
11013
none\11014.jpg 4 10
11014
none\11015.jpg 4 10
11015
none\11016.jpg 4 10
11016
none\11017.jpg 4 10
11017
none\11018.jpg 4 10
11018
none\11019.jpg 4 10
11019
none\1102.jpg 4 9
1102
none\11020.jpg 4 10
11020
none\11021.jpg 4 10
11021
none\11022.jpg 4 10
11022
none\11023.jpg 4 10
11023
none\11024.jpg 4 10
11024
none\11025.jpg 4 10
11025
none\11026.jpg 4 10
11026
none\11027.jpg 4 10
11027
none\11028.jpg 4 10
11028
none\11

none\11557.jpg 4 10
11557
none\11558.jpg 4 10
11558
none\11559.jpg 4 10
11559
none\1156.jpg 4 9
1156
none\11560.jpg 4 10
11560
none\11561.jpg 4 10
11561
none\11562.jpg 4 10
11562
none\11563.jpg 4 10
11563
none\11564.jpg 4 10
11564
none\11565.jpg 4 10
11565
none\11566.jpg 4 10
11566
none\11567.jpg 4 10
11567
none\11568.jpg 4 10
11568
none\11569.jpg 4 10
11569
none\1157.jpg 4 9
1157
none\11570.jpg 4 10
11570
none\11571.jpg 4 10
11571
none\11572.jpg 4 10
11572
none\11573.jpg 4 10
11573
none\11574.jpg 4 10
11574
none\11575.jpg 4 10
11575
none\11576.jpg 4 10
11576
none\11577.jpg 4 10
11577
none\11578.jpg 4 10
11578
none\11579.jpg 4 10
11579
none\1158.jpg 4 9
1158
none\11580.jpg 4 10
11580
none\11581.jpg 4 10
11581
none\11582.jpg 4 10
11582
none\11583.jpg 4 10
11583
none\11584.jpg 4 10
11584
none\11585.jpg 4 10
11585
none\11586.jpg 4 10
11586
none\11587.jpg 4 10
11587
none\11588.jpg 4 10
11588
none\11589.jpg 4 10
11589
none\1159.jpg 4 9
1159
none\11590.jpg 4 10
11590
none\11591.jpg 4 10
1159

...

none\7956.jpg 4 9
7956
none\7957.jpg 4 9
7957
none\7958.jpg 4 9
7958
none\7959.jpg 4 9
7959
none\796.jpg 4 8
796
none\7960.jpg 4 9
7960
none\7961.jpg 4 9
7961
none\7962.jpg 4 9
7962
none\7963.jpg 4 9
7963
none\7964.jpg 4 9
7964
none\7965.jpg 4 9
7965
none\7966.jpg 4 9
7966
none\7967.jpg 4 9
7967
none\7968.jpg 4 9
7968
none\7969.jpg 4 9
7969
none\797.jpg 4 8
797
none\7970.jpg 4 9
7970
none\7971.jpg 4 9
7971
none\7972.jpg 4 9
7972
none\7973.jpg 4 9
7973
none\7974.jpg 4 9
7974
none\7975.jpg 4 9
7975
none\7976.jpg 4 9
7976
none\7977.jpg 4 9
7977
none\7978.jpg 4 9
7978
none\7979.jpg 4 9
7979
none\798.jpg 4 8
798
none\7980.jpg 4 9
7980
none\7981.jpg 4 9
7981
none\7982.jpg 4 9
7982
none\7983.jpg 4 9
7983
none\7984.jpg 4 9
7984
none\7985.jpg 4 9
7985
none\7986.jpg 4 9
7986
none\7987.jpg 4 9
7987
none\7988.jpg 4 9
7988
none\7989.jpg 4 9
7989
none\799.jpg 4 8
799
none\7990.jpg 4 9
7990
none\7991.jpg 4 9
7991
none\7992.jpg 4 9
7992
none\7993.jpg 4 9
7993
none\7994.jpg 4 9
7994
none\7995.jpg 4 9
7

none\8630.jpg 4 9
8630
none\8631.jpg 4 9
8631
none\8632.jpg 4 9
8632
none\8633.jpg 4 9
8633
none\8634.jpg 4 9
8634
none\8635.jpg 4 9
8635
none\8636.jpg 4 9
8636
none\8637.jpg 4 9
8637
none\8638.jpg 4 9
8638
none\8639.jpg 4 9
8639
none\864.jpg 4 8
864
none\8640.jpg 4 9
8640
none\8641.jpg 4 9
8641
none\8642.jpg 4 9
8642
none\8643.jpg 4 9
8643
none\8644.jpg 4 9
8644
none\8645.jpg 4 9
8645
none\8646.jpg 4 9
8646
none\8647.jpg 4 9
8647
none\8648.jpg 4 9
8648
none\8649.jpg 4 9
8649
none\865.jpg 4 8
865
none\8650.jpg 4 9
8650
none\8651.jpg 4 9
8651
none\8652.jpg 4 9
8652
none\8653.jpg 4 9
8653
none\8654.jpg 4 9
8654
none\8655.jpg 4 9
8655
none\8656.jpg 4 9
8656
none\8657.jpg 4 9
8657
none\8658.jpg 4 9
8658
none\8659.jpg 4 9
8659
none\866.jpg 4 8
866
none\8660.jpg 4 9
8660
none\8661.jpg 4 9
8661
none\8662.jpg 4 9
8662
none\8663.jpg 4 9
8663
none\8664.jpg 4 9
8664
none\8665.jpg 4 9
8665
none\8666.jpg 4 9
8666
none\8667.jpg 4 9
8667
none\8668.jpg 4 9
8668
none\8669.jpg 4 9
8669
none\867.jpg 4 8


none\9305.jpg 4 9
9305
none\9306.jpg 4 9
9306
none\9307.jpg 4 9
9307
none\9308.jpg 4 9
9308
none\9309.jpg 4 9
9309
none\931.jpg 4 8
931
none\9310.jpg 4 9
9310
none\9311.jpg 4 9
9311
none\9312.jpg 4 9
9312
none\9313.jpg 4 9
9313
none\9314.jpg 4 9
9314
none\9315.jpg 4 9
9315
none\9316.jpg 4 9
9316
none\9317.jpg 4 9
9317
none\9318.jpg 4 9
9318
none\9319.jpg 4 9
9319
none\932.jpg 4 8
932
none\9320.jpg 4 9
9320
none\9321.jpg 4 9
9321
none\9322.jpg 4 9
9322
none\9323.jpg 4 9
9323
none\9324.jpg 4 9
9324
none\9325.jpg 4 9
9325
none\9326.jpg 4 9
9326
none\9327.jpg 4 9
9327
none\9328.jpg 4 9
9328
none\9329.jpg 4 9
9329
none\933.jpg 4 8
933
none\9330.jpg 4 9
9330
none\9331.jpg 4 9
9331
none\9332.jpg 4 9
9332
none\9333.jpg 4 9
9333
none\9334.jpg 4 9
9334
none\9335.jpg 4 9
9335
none\9336.jpg 4 9
9336
none\9337.jpg 4 9
9337
none\9338.jpg 4 9
9338
none\9339.jpg 4 9
9339
none\934.jpg 4 8
934
none\9340.jpg 4 9
9340
none\9341.jpg 4 9
9341
none\9342.jpg 4 9
9342
none\9343.jpg 4 9
9343
none\9344.jpg 4 9
9

none\9981.jpg 4 9
9981
none\9982.jpg 4 9
9982
none\9983.jpg 4 9
9983
none\9984.jpg 4 9
9984
none\9985.jpg 4 9
9985
none\9986.jpg 4 9
9986
none\9987.jpg 4 9
9987
none\9988.jpg 4 9
9988
none\9989.jpg 4 9
9989
none\999.jpg 4 8
999
none\9990.jpg 4 9
9990
none\9991.jpg 4 9
9991
none\9992.jpg 4 9
9992
none\9993.jpg 4 9
9993
none\9994.jpg 4 9
9994
none\9995.jpg 4 9
9995
none\9996.jpg 4 9
9996
none\9997.jpg 4 9
9997
none\9998.jpg 4 9
9998
none\9999.jpg 4 9
9999


,id,label
0,1,0.995
1,2,0.995
2,3,0.995
3,4,0.995
4,5,0.005
5,6,0.005
6,7,0.005
7,8,0.005
8,9,0.005
9,10,0.005
